In [1]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import xgboost as xgb
import time
import itertools
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [2]:
# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

print(df_All.head())

   425_pct_Al  M_Value  C4_pct_Eth  C4_pct_H2O  C4_pct_Hex  \
0    6.307630  3.50893    2.508420    21.96340    0.659256   
1    6.308335  3.50863    2.511135    21.96865    0.659930   
2    6.309390  3.50818    2.515210    21.97655    0.660942   
3    6.310090  3.50788    2.517925    21.98185    0.661616   
4    6.310790  3.50758    2.520640    21.98715    0.662291   

   HydWtr_pct_Ammonia  HydWtr_Na2O   DI55152   FC55003       FC55552  \
0            0.908765     2.712180  0.925255  4919.290  41564.100000   
1            0.907643     2.805415  0.924281  5039.445  41558.250000   
2            0.905959     2.945270  0.923773  5268.620  41524.066667   
3            0.904837     3.038505  0.924103  5349.320  41500.275000   
4            0.903715     3.131740  0.924258  5563.165  41466.625000   

       FC55569  FFC55553  FFC55555    LC55555   PI55004   TC55552     TC55555  \
0  6918.110000  0.996975  0.751501  59.968800  1.306310  177.1880  180.125000   
1  6919.960000  0.998847  0.7505

In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# List of columns to exclude to run XGboost feature selection
exclude_columns = [#'Date',
                 'C4_pct_Hex', 'HydWtr_Na2O',
                 'TC55555'  
                   ]

# Create a new DataFrame without the excluded columnsd
df_All = df_All.drop(columns=exclude_columns)

In [5]:
print(df_All.describe())

         425_pct_Al       M_Value    C4_pct_Eth    C4_pct_H2O  \
count  54701.000000  54701.000000  54701.000000  54701.000000   
mean       6.112584      3.604726      1.274754     20.756146   
std        0.316017      0.173736      0.677477      2.859486   
min        0.252306      1.461875      0.086948      7.932310   
25%        5.987750      3.506780      0.675285     18.331550   
50%        6.130470      3.595655      1.291115     20.731850   
75%        6.270075      3.696090      1.719630     22.989150   
max        8.020685      5.827565      8.711360     33.731050   

       HydWtr_pct_Ammonia       DI55152       FC55003       FC55552  \
count        54701.000000  54701.000000  54701.000000  54701.000000   
mean             0.969410      0.944165   5946.323971  36128.859598   
std              0.144668      0.042737    794.065219   4270.390546   
min              0.390025      0.815428   2991.532500  18800.350000   
25%              0.877716      0.915431   5446.200000  3499

In [6]:
# Splitting into train and test
X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
y = df_All['Butanol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the parameter grid

param_grid = {
    'learning_rate': [0.03, 0.05, 0.1, 0.2],  # Introduced some intermediate and higher values.
    'n_estimators': [800],     # Expanded around 500, as it was your best result.
    'max_depth': [9],                # Introduced values around 7, which was your best depth. 6, 7, 8, 
    'subsample': [0.9],        # Expanded a bit to include values around 0.8.   0.7, 0.8, , 1.0
    'colsample_bytree': [0.8],      # Values around your best result of 0.7.  0.6, 0.7, 
    'gamma': [0.5],                   # Regularization parameter. Start with these values. 0, 0.1, 
    'alpha': [0.1],                   # L1 regularization term. 0, , 0.5
    'lambda': [1],                    # L2 regularization term.  , 1.5, 2
    'colsample_bylevel': [0.8],     # Additional column sampling.  0.6, 0.7, 
    'colsample_bynode': [ 0.7]       # More granular column sampling  .0.6, 0.8
}

# param_grid = {
#     'learning_rate': [0.01, 0.1],
#     'n_estimators': [100, 500],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.3, 0.5, 0.7]
# }
# Best Parameters: {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.7}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', use_label_encoder=False)

def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

best_score = float('inf')
best_params = None
results = []

# Loop over all combinations of hyperparameters
for params in product_dict(**param_grid):
    start_time = time.time()
    
    xgb_reg.set_params(**params)
    y_pred = cross_val_predict(xgb_reg, X_train_scaled, y_train, cv=3)

    mse = mean_squared_error(y_train, y_pred)
    mae = mean_absolute_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train_scaled.shape[1]-1)
    rmse = np.sqrt(mse)

    end_time = time.time()
    duration = end_time - start_time

    print(f"Evaluating {params} took {duration:.2f} seconds")
    print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R^2: {r2:.4f}, Adjusted R^2: {adj_r2:.4f}, RMSE: {rmse:.4f}")

    results.append({
        'Parameters': params,
        'MSE': mse,
        'MAE': mae,
        'R^2': r2,
        'Adjusted R^2': adj_r2,
        'RMSE': rmse,
        'Duration': duration
    })

    if mse < best_score:
        best_score = mse
        best_params = params

results_df = pd.DataFrame(results)

print("Best Parameters:", best_params)


Evaluating {'learning_rate': 0.03, 'n_estimators': 800, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.5, 'alpha': 0.1, 'lambda': 1, 'colsample_bylevel': 0.8, 'colsample_bynode': 0.7} took 22.82 seconds
MSE: 6.1106, MAE: 1.2995, R^2: 0.9297, Adjusted R^2: 0.9297, RMSE: 2.4720
Evaluating {'learning_rate': 0.05, 'n_estimators': 800, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.5, 'alpha': 0.1, 'lambda': 1, 'colsample_bylevel': 0.8, 'colsample_bynode': 0.7} took 19.58 seconds
MSE: 5.5242, MAE: 1.2149, R^2: 0.9364, Adjusted R^2: 0.9364, RMSE: 2.3504
Evaluating {'learning_rate': 0.1, 'n_estimators': 800, 'max_depth': 9, 'subsample': 0.9, 'colsample_bytree': 0.8, 'gamma': 0.5, 'alpha': 0.1, 'lambda': 1, 'colsample_bylevel': 0.8, 'colsample_bynode': 0.7} took 11.69 seconds
MSE: 5.7004, MAE: 1.2460, R^2: 0.9344, Adjusted R^2: 0.9344, RMSE: 2.3876
Evaluating {'learning_rate': 0.2, 'n_estimators': 800, 'max_depth': 9, 'subsample': 0.9, 'colsample_b

In [7]:
results_df = pd.DataFrame(results)

In [8]:
print(results_df.head())

# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
results_df.to_csv('results.csv', index=False)

                                          Parameters       MSE       MAE  \
0  {'learning_rate': 0.03, 'n_estimators': 800, '...  6.110560  1.299550   
1  {'learning_rate': 0.05, 'n_estimators': 800, '...  5.524242  1.214944   
2  {'learning_rate': 0.1, 'n_estimators': 800, 'm...  5.700427  1.245971   
3  {'learning_rate': 0.2, 'n_estimators': 800, 'm...  6.490183  1.367121   

        R^2  Adjusted R^2      RMSE   Duration  
0  0.929692      0.929668  2.471955  22.818402  
1  0.936438      0.936417  2.350371  19.577923  
2  0.934411      0.934389  2.387557  11.686263  
3  0.925324      0.925299  2.547584   7.987237  
